In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

realCIR = np.load('real.npy')
imagCIR = np.load('imag.npy')
magCIR = np.load('dataset.npy')
GT = np.load('label.npy')

print(realCIR.shape, imagCIR.shape, magCIR.shape, GT.shape)

tan = np.arctan2(imagCIR, realCIR)
diff = tan[:,:63] - tan[:, 1:]
diffrence = np.zeros((magCIR.shape[0],magCIR.shape[1]))
diffrence[:,1:] = diff

array = np.stack([magCIR,diffrence],axis=-1)
# array = np.stack([magCIR,tan],axis=-1)

X_train, X_test, y_train, y_test = train_test_split(array, GT, test_size=0.2, shuffle=True, stratify=GT)

(33000, 64) (33000, 64) (33000, 64) (33000,)


In [2]:
model = tf.keras.models.Sequential([
    # layer1
    tf.keras.layers.Conv1D(96,11, activation=tf.nn.relu,input_shape=(64,2),padding='same'),
    tf.keras.layers.MaxPooling1D(2,padding='same'),
    # layer2
    tf.keras.layers.Conv1D(256,5, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.MaxPooling1D(2,padding='same'),
    # layer3
    tf.keras.layers.ZeroPadding1D(1),
    tf.keras.layers.Conv1D(384,3, activation=tf.nn.relu,padding='same'),
    # layer4
    tf.keras.layers.ZeroPadding1D(1),
    tf.keras.layers.Conv1D(384,3, activation=tf.nn.relu,padding='same'),
    # layer5
    tf.keras.layers.ZeroPadding1D(1),
    tf.keras.layers.Conv1D(256,3, activation=tf.nn.relu,padding='same'),
    tf.keras.layers.Dropout(0.5),
    # layer6
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # layer7
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # layer8
    tf.keras.layers.Dense(6, activation='softmax'),
])

2022-11-29 16:26:06.167871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def step_decay(epoch):
    start = 0.0001
    drop = 0.1
    epochs_drop = 5
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

In [4]:
lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

In [5]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train, epochs=10,
            validation_data=(X_test, y_test),
             callbacks=[lr_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/10
825/825 [==============================] - 238s 287ms/step - loss: 3.1087 - accuracy: 0.3394 - val_loss: 1.2915 - val_accuracy: 0.4752 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001.
Epoch 2/10
825/825 [==============================] - 239s 290ms/step - loss: 1.3400 - accuracy: 0.4417 - val_loss: 1.1243 - val_accuracy: 0.5400 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/10
825/825 [==============================] - 242s 294ms/step - loss: 1.2111 - accuracy: 0.4977 - val_loss: 0.9964 - val_accuracy: 0.5995 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/10
825/825 [==============================] - 255s 310ms/step - loss: 1.0991 - accuracy: 0.5450 - val_loss: 0.8972 - val_accuracy: 0.6380 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0001.
Epoch 5/10
825/825 [===

In [7]:
model.evaluate(X_test, y_test)

207/207 [==============================] - 9s 41ms/step - loss: 0.6778 - accuracy: 0.7302


[0.6778316497802734, 0.7301515340805054]